In [1]:
import pandas as pd
import os
from tqdm import tqdm
!pip install absl-py nltk numpy six>=1.14 pythainlp pyonmttok jieba fugashi[unidic]

In [2]:
language='telugu'
lang_code = "te"
actual_fp='/kaggle/input/tesum-dataset/TeSum_test_data.csv'
predictions_fp='/kaggle/input/tesum-checkpoint-5-epochs/model_checkpoints/generated_predictions.txt'

In [3]:
df=pd.read_csv(actual_fp)
df.head(1)

,id,url,title,cleaned_text,summary,article_sentence_count,article_token_count,summary_token_count,title_token_count,compression,abstractivity
0,18081,https://www.prajasakti.com/WEBSECTION/National...,పోర్చుగీసు ప్రధానితో మోడీ భేటీ.\n,భారత ప్రధాన మంత్రి నరేంద్ర మోడీ కొద్ది సేపటి క...,భారత ప్రధాన మంత్రి నరేంద్ర మోడీ నేడు పోర్చుగీస...,4,36,18,6,50.0,16.67


In [4]:
actual_labels=list(df['summary'].values)

with open(predictions_fp) as fp:
    predicted_labels=fp.readlines()
    predicted_labels=[i.strip() for i in predicted_labels]

if len(actual_labels)==len(predicted_labels):
    print('Sizes Matched')
else:
    print('WARNING: Actual and Predicted Sizes Not Matched')

Sizes Matched


In [5]:
len(df), len(predicted_labels)

(2017, 2017)

# Multilingual ROUGE

In [6]:
os.chdir('/kaggle/input/multi-lingual-rouge-score/multilingual rouge')
from rouge_score import rouge_scorer
os.chdir('/kaggle/working')

In [7]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False, lang=language)

In [8]:
def return_rouge_score(ref,hyp):
    scores = scorer.score(ref, hyp) # ref and hyp must be in string format only
    r1_f=scores['rouge1'][2] # index 2 corresponds to f-1 score, 0-precsion,1-recall
    r2_f=scores['rouge2'][2]
    rL_f=scores['rougeL'][2]
    
    return r1_f,r2_f,rL_f

In [9]:
r1=[]
r2=[]
rL=[]

references=actual_labels
hypothesis=predicted_labels

In [10]:
for i in range(len(actual_labels)):
#     r1_f,r2_f,rL_f=return_rouge_score(get_encoded(references[i]),get_encoded(hypothesis[i]))
    r1_f,r2_f,rL_f=return_rouge_score(str(references[i]),str(hypothesis[i]))
    
    r1.append(r1_f)
    r2.append(r2_f)
    rL.append(rL_f)

In [11]:
df['Prediction']=predicted_labels
df['ROUGE-1']=r1
df['ROUGE-2']=r2
df['ROUGE-L']=rL

In [12]:
df.sort_values(by=['ROUGE-L'],inplace=True,ascending=False)

In [13]:
df.head(1)

,id,url,title,cleaned_text,summary,article_sentence_count,article_token_count,summary_token_count,title_token_count,compression,abstractivity,Prediction,ROUGE-1,ROUGE-2,ROUGE-L
11,37579,https://www.prajasakti.com/WEBSECTION/Internat...,నూతన అదనపు భవనాల నిర్మాణ ప్రక్రియ.,విజయనగరం జిల్లా బొబ్బిలి పట్టణంలోని మున్సిఫ్ మ...,విజయనగరం జిల్లా బొబ్బిలి పట్టణంలోని మున్సిఫ్ మ...,4,95,38,6,60.0,10.53,విజయనగరం జిల్లా బొబ్బిలి పట్టణంలోని మున్సిఫ్ మ...,0.84507,0.724638,0.84507


In [14]:
df[['ROUGE-1','ROUGE-2','ROUGE-L']].describe()

,ROUGE-1,ROUGE-2,ROUGE-L
count,2017.000000,2017.000000,2017.000000
mean,0.387859,0.224620,0.321154
std,0.129061,0.120668,0.121375
min,0.000000,0.000000,0.000000
25%,0.295652,0.132231,0.231579
50%,0.387097,0.215385,0.315789
75%,0.480769,0.304000,0.400000
max,0.845070,0.731707,0.845070
